<a href="https://colab.research.google.com/github/shriyaaduvala/Detecting-DDOS-Attack/blob/main/DDOS_Attack.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import math
import random
from collections import Counter
from sklearn.metrics import accuracy_score

In [2]:
def generate_packets(packet_count, attack=False):
    """
    Simulate data packet generation.
    :param packet_count: Number of packets to generate.
    :param attack: If True, simulate attack traffic.
    :return: List of IP addresses.
    """
    packets = []
    if attack:
        # Simulate DDoS attack by heavily using one source IP
        attacker_ip = f"192.168.1.{random.randint(1, 255)}"
        packets = [attacker_ip for _ in range(packet_count)]
    else:
        # Generate packets from random IPs
        for _ in range(packet_count):
            ip = f"192.168.{random.randint(0, 255)}.{random.randint(1, 255)}"
            packets.append(ip)
    return packets

In [3]:
def compute_entropy(packets):
    """
    Calculate the entropy of packet sources.
    :param packets: List of IP addresses.
    :return: Entropy value.
    """
    total_packets = len(packets)
    if total_packets == 0:
        return 0
    counter = Counter(packets)
    probabilities = [count / total_packets for count in counter.values()]
    entropy = -sum(p * math.log2(p) for p in probabilities)
    return entropy

In [4]:
def detect_ddos(entropy, threshold=2.5):
    """
    Detect if traffic is a DDoS attack based on entropy.
    :param entropy: Computed entropy.
    :param threshold: Threshold for entropy.
    :return: True if DDoS detected, False otherwise.
    """
    return entropy < threshold

In [5]:
# Generate datasets
normal_packets = generate_packets(1000, attack=False)
attack_packets = generate_packets(1000, attack=True)

# Calculate entropies
normal_entropy = compute_entropy(normal_packets)
attack_entropy = compute_entropy(attack_packets)

# Predict outcomes
normal_detection = detect_ddos(normal_entropy)
attack_detection = detect_ddos(attack_entropy)

# Evaluate accuracy using a simple labeled dataset
true_labels = [0] * 1000 + [1] * 1000  # 0 for normal, 1 for attack
predicted_labels = [detect_ddos(compute_entropy(generate_packets(100, attack=False))) for _ in range(1000)] + \
                   [detect_ddos(compute_entropy(generate_packets(100, attack=True))) for _ in range(1000)]

accuracy = accuracy_score(true_labels, predicted_labels)
print(f"Accuracy of DDoS Detection: {accuracy * 100:.2f}%")

# Print entropy values and detection results
print(f"Normal Traffic Entropy: {normal_entropy:.2f}, Detected as DDoS: {normal_detection}")
print(f"Attack Traffic Entropy: {attack_entropy:.2f}, Detected as DDoS: {attack_detection}")

Accuracy of DDoS Detection: 100.00%
Normal Traffic Entropy: 9.96, Detected as DDoS: False
Attack Traffic Entropy: -0.00, Detected as DDoS: True
